# **Imports**

In [ ]:
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.5.2

In [ ]:
import sklearn as skl
print(f"New scikit-learn version: {skl.__version__}")
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from numpy import median
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

RANDOM_SEED = 1507

# **Datasets**

In [ ]:
data_train = pd.read_csv('/content/sample_data/train_radiomics_hipocamp.csv')
data_test = pd.read_csv('/content/sample_data/test_radiomics_hipocamp.csv')
data_occipital = pd.read_csv('/content/sample_data/train_radiomics_occipital_CONTROL.csv')
data_occipital.head()

# **Análise de Dados**

## Verificação de Missing Values

In [ ]:
missing_train = data_train.isna().sum()
total_missing_train = missing_train.sum()
print(f"Total de missing values no dataset de treino: {total_missing_train}")

missing_test = data_test.isna().sum()
total_missing_test = missing_test.sum()
print(f"\nTotal de missing values no dataset de teste: {total_missing_test}")

## Veirifição das Colunas com o Mesmo Valor em Todas as Entradas

In [ ]:
colunas_mesmos_valores_train = data_train.loc[:, data_train.nunique() == 1].columns
print("Colunas com todas as entradas iguais no dataset de treino:")
print(colunas_mesmos_valores_train)

colunas_mesmos_valores_test = data_test.loc[:, data_test.nunique() == 1].columns
print("Colunas com todas as entradas iguais no dataset de teste:")
print(colunas_mesmos_valores_test)

## Verificação de Linhas Duplicadas

In [ ]:
linhas_duplicadas_train = data_train.duplicated().sum()
print(f"Número de linhas duplicadas no dataset treino: {linhas_duplicadas_train}")

linhas_duplicadas_test = data_test.duplicated().sum()
print(f"Número de linhas duplicadas no dataset teste: {linhas_duplicadas_test}")

## Verificação de Colunas com Entradas Alfanuméricas

In [ ]:
colunas_categoricas_train = data_train.select_dtypes(include=['object', 'category']).shape[1]
print(f"Colunas categóricas no dataset treino: {colunas_categoricas_train}")

colunas_categoricas_test = data_test.select_dtypes(include=['object', 'category']).shape[1]
print(f"Colunas categóricas no dataset teste: {colunas_categoricas_test}")

## Verificação de Valores *Outliers*

In [ ]:
# Calcular IQR para cada coluna
Q1 = data_train.quantile(0.25)
Q3 = data_train.quantile(0.75)
IQR = Q3 - Q1

# Determinar os limites inferior e superior
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Verificar outliers
outliers = (data_train < lower_bound) | (data_train > upper_bound)
print(outliers.sum())  # Número de outliers por coluna

## *Sex* - Gráfico de Barras

In [ ]:
contagem_sex = data_train['Sex'].value_counts()

# Definir rótulos e valores
labels = ['Homens', 'Mulheres']  # Ajusta conforme os valores na coluna 'Sex'
valores = contagem_sex.values

# Criar o gráfico de barras
plt.figure(figsize=(8, 6))  # Tamanho do gráfico
plt.bar(labels, valores, color=['blue', 'pink'], edgecolor='black')

# Adicionar rótulos e título
plt.xlabel("Género", fontsize=12)
plt.ylabel("Quantidade", fontsize=12)
plt.title("Distribuição de Homens e Mulheres", fontsize=14)

# Adicionar os valores em cima das barras
for i, v in enumerate(valores):
    plt.text(i, v + 0.5, str(v), ha='center', fontsize=10)

# Mostrar o gráfico
plt.tight_layout()  # Ajustar o layout
plt.show()

## *Sex* - Pie Chart

In [ ]:
contagem_sex = data_train['Sex'].value_counts()


labels = ['Homens', 'Mulheres']

plt.figure(figsize=(8, 8))
plt.pie(contagem_sex, labels=labels, autopct='%1.1f%%', startangle=90, colors=['blue', 'pink'])
plt.title("Distribuição de Homens e Mulheres")
plt.show()

## *Transition* - Gráfico de Barras

In [ ]:
contagem_transitions = data_train['Transition'].value_counts()

plt.figure(figsize=(10, 6))  # Tamanho do gráfico
plt.bar(contagem_transitions.index, contagem_transitions.values, color='skyblue', edgecolor='black')

plt.xlabel("Categorias em 'Transition'", fontsize=12)
plt.ylabel("Quantidade", fontsize=12)
plt.title("Distribuição das Categorias em 'Transition'", fontsize=14)
plt.xticks(rotation=45)  # Rodar os rótulos para melhor legibilidade
plt.tight_layout()  # Ajustar layout para evitar cortes

for i, v in enumerate(contagem_transitions.values):
    plt.text(i, v + 0.5, str(v), ha='center', fontsize=10)

plt.show()

## *Transition* - Pie Chart

In [ ]:
contagem_transitions = data_train['Transition'].value_counts()

plt.figure(figsize=(10, 6))  # Tamanho do gráfico
plt.pie(contagem_transitions, labels=contagem_transitions.index, autopct='%1.1f%%', startangle=90)
plt.title("Distribuição das Categorias em 'Transition'")
plt.show()

## Relação entre *Sex* e *Transition*

In [ ]:
# Criar o gráfico de barras agrupadas
plt.figure(figsize=(10, 6))  # Tamanho do gráfico
sns.countplot(data=data_train, x='Transition', hue='Sex', palette='viridis')

# Adicionar rótulos e título
plt.xlabel("Transition", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.title("Relação entre Sex e Transition", fontsize=14)
plt.legend(title="Sex", labels=["Mulheres", "Homens"])
plt.tight_layout()

# Mostrar o gráfico
plt.show()

# Contar os valores 0 e 1 na coluna 'Sex'
contagem_sex = data_train['Sex'].value_counts()

# **Tratamento de Dados**

## Remoção das Colunas com o Mesmo Valor em Todas as Entradas

In [ ]:
data_train = data_train.loc[:, (data_train.nunique() > 1)]
data_test = data_test.loc[:, (data_test.nunique() > 1)]

## Remoção de Colunas com Entradas Alfanuméricas

In [ ]:
columns_to_drop = ["ID","Image", "Mask",'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash',
                   'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex',
                   'diagnostics_Mask-original_CenterOfMass']

data_train = data_train.drop(columns=columns_to_drop)
data_test = data_test.drop(columns=columns_to_drop)

## Converter a Coluna *\[Age\]* para o tipo INT

In [ ]:
data_train['Age'] = data_train['Age'].astype(int)
data_test['Age'] = data_test['Age'].astype(int)

## Converter a Coluna *\[Transition\]* para valores Numéricos

In [ ]:
data_train["Transition"] = data_train['Transition'].replace({'CN-CN':0,'CN-MCI':1,'MCI-MCI':2,'MCI-AD':3,'AD-AD':4}).astype(int)

## Remoção de Valores *Outliers*

In [ ]:
# Substituir valores fora dos limites
data_train = data_train.clip(lower=lower_bound, upper=upper_bound, axis=1)
data_test = data_test.clip(lower=lower_bound, upper=upper_bound, axis=1)

outliers = (data_train < lower_bound) | (data_train > upper_bound)
print(outliers.sum())  # Número de outliers por coluna

# **Análise do *Dataset* Tratado**

In [ ]:
num_linhas_train = data_train.shape[0]
print(f"Número de linhas no dataset de treino: {num_linhas_train}")

num_colunas_train = data_train.shape[1]
print(f"Número de colunas no dataset de treino: {num_colunas_train}")

num_linhas_test = data_test.shape[0]
print(f"Número de linhas no dataset de teste: {num_linhas_test}")

num_colunas_test = data_test.shape[1]
print(f"Número de colunas no dataset de teste: {num_colunas_test}")

data_train.head()

# **Feature Selection**

In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

X_treino = data_train.drop('Transition', axis=1)
y_treino = data_train['Transition']

X_teste = data_test


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_treino, y_treino, test_size=0.3, random_state=2023, stratify=y_treino)

# **Modelação**

## **Random Forest Classifier**

In [ ]:
from sklearn.metrics import make_scorer, f1_score

# Definir o modelo Random Forest
rf = RandomForestClassifier(random_state=RANDOM_SEED)

scorer = make_scorer(f1_score, average='macro')

# Definir o grid de parâmetros
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10],
    'max_features': ['sqrt', 'log2'],  # Corrigir para 'sqrt' ou 'log2'
    'bootstrap': [True, False],
    'class_weight': ['balanced', None]  # Para tratar desbalanceamento de classes
}

# Usar GridSearchCV para encontrar os melhores parâmetros
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring=scorer, cv=4, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Exibir os melhores parâmetros encontrados
print("Melhores parâmetros encontrados: ", grid_search.best_params_)

# Usar os melhores parâmetros para fazer previsões
best_rf = grid_search.best_estimator_
predictions = best_rf.predict(X_test)

# Avaliar a performance
print(classification_report(y_test, predictions))

## MLP

In [ ]:
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
import random
from itertools import product
import torch

# Fixar a seed
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Escalando os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convertendo os dados para tensores do PyTorch
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test.values)

# Definindo o modelo MLP classificado
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.bn1 = nn.BatchNorm1d(hidden_sizes[0])  # Batch Normalization
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.bn2 = nn.BatchNorm1d(hidden_sizes[1])  # Batch Normalization
        self.fc3 = nn.Linear(hidden_sizes[1], output_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x  # Sem Softmax

# Definir hiperparâmetros para o GridSearch
param_grid = {
    'hidden_sizes': [[256, 128], [128, 64], [64, 32]],
    'lr': [0.001, 0.005, 0.01],
    'dropout_rate': [0.2, 0.3, 0.4],
    'weight_decay': [1e-4, 1e-5]
}

# Criar combinações de hiperparâmetros
combinations = list(product(*param_grid.values()))
best_accuracy = 0
best_params = None

# Loop para GridSearch
for params in combinations:
    hidden_sizes, lr, dropout_rate, weight_decay = params
    model = MLP(X_train.shape[1], hidden_sizes, len(set(y_train)), dropout_rate)

    # Inicializar a função de perda e o otimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)

    # Treinamento do modelo
    epochs = 100
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

    # Avaliação do modelo
    model.eval()
    with torch.no_grad():
        y_pred_tensor = model(X_test_tensor)
        y_pred = torch.argmax(y_pred_tensor, axis=1).numpy()

    # Calcular a accuracy
    acc = accuracy_score(y_test, y_pred)
    print(f"Params: {params}, Accuracy: {acc:.4f}")

    # Verificar se é a melhor accuracy
    if acc > best_accuracy:
        best_accuracy = acc
        best_params = params

# Imprimir os melhores parâmetros
print("\nMelhores Parâmetros:", best_params)
print("Melhor Accuracy:", best_accuracy)

In [ ]:
# Utilizar os melhores hiperparâmetros encontrados
hidden_sizes = [64, 32]
lr = 0.001
dropout_rate = 0.3
weight_decay = 1e-5

# Inicializar o modelo com os melhores parâmetros
model = MLP(X_train.shape[1], hidden_sizes, len(set(y_train)), dropout_rate)

# Inicializar a função de perda e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)

# Treinamento do modelo
epochs = 300  # Número de épocas pode ser ajustado
train_losses = []
test_losses = []

best_val_loss = float('inf')
patience = 30
trigger_times = 0

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Salvar a perda de treinamento
    train_losses.append(loss.item())

    # Modo de validação
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_test_tensor)
        val_loss = criterion(val_outputs, y_test_tensor)
        test_losses.append(val_loss.item())

    # Early stopping
    if val_loss.item() < best_val_loss:
        best_val_loss = val_loss.item()
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f"Early stopping at epoch {epoch + 1}")
            break

    # Log de progresso
    if (epoch + 1) % 20 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {loss.item():.4f}, Test Loss: {val_loss.item():.4f}")

# Avaliação do modelo
model.eval()
with torch.no_grad():
    y_pred_tensor = model(X_test_tensor)
    y_pred = torch.argmax(y_pred_tensor, axis=1).numpy()

# Relatório de classificação
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))

## XGBoost

In [ ]:
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Criar o modelo de XGBoost
xgb = XGBClassifier(random_state=2023, objective='multi:softprob', num_class=5)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=2023)

param_grid = {
    'learning_rate': [0.005],
    'n_estimators': [1000],
    'max_depth': [8],
    'gamma': [0.1],
    'min_child_weight': [1],
    'colsample_bytree': [1.0],
    'reg_alpha': [0.2],
    'reg_lambda': [2]
}

"""
Valores para o mitigar o overfit
param_grid = {
  'learning_rate': [0.005],
  'n_estimators': [1000],
  'max_depth': [4, 5],
  'gamma': [0.1, 0.3],
  'min_child_weight': [1, 3],
  'colsample_bytree': [0.8],
  'subsample': [0.8],
  'reg_alpha': [0.2, 1.0],
  'reg_lambda': [2, 5]
}"""

"""
Valores para o melhor score privado
param_grid = {
    'learning_rate': [0.01],
    'n_estimators': [800],
    'max_depth': [5],
    'gamma': [0.1],
    'min_child_weight': [1],
    'colsample_bytree': [1.0],
}"""

grid_searchXGB = GridSearchCV(xgb, param_grid, cv=cv, refit=True, verbose=3,return_train_score=True)
grid_searchXGB.fit(X_train, y_train)
grid_predictionXGB = grid_searchXGB.predict(data_test)

print(grid_searchXGB.best_estimator_)

XGB_best = grid_searchXGB.best_estimator_

# **Converter para *CSV***

In [ ]:
submission = pd.DataFrame(grid_predictionXGB, columns = ["Transition"])
submission.insert(0, "RowId", range(1,len(grid_predictionXGB) + 1), True)

print(submission)

#transformação dos valores para formato escrito
submission['Transition']= submission['Transition'].replace({0 : 'CN-CN', 1 : 'CN-MCI', 2 : 'MCI-MCI', 3 : 'MCI-AD', 4 : 'AD-AD'})

print(submission)

#passagem para ficheiro csv
submission.to_csv('submission_Kaggle.csv', index=False)